In the previous section we introduced `Enum` and `Sum`.   In this section we will give a more thoruogh introduction ADTs.

ADTs in peak do not have a fixed representation in bitvectors.  Instead this representation is later defined seperately with an *assembler*.  This allows a designer to fix the semenatics procossor with respect an abstract instruction without defining the exact bit level encoding.  This makes it simple to extend the instruction set as one need not edit existing code.

ADTs are one of the most complex parts of peak for they

In [1]:
import hwtypes as hw
from hwtypes.adt import Enum, Product, TaggedUnion, Sum, Tuple

Enums define a set of named abstract values.  They are inherited from `hw.Enum`.

Enum members may be assigned a value which acts as suggestion to the assembler or may be left entirely abstract using `hw.Enum.Auto` or its alias `new_instruction`.  Enum members suppot equals and not equals.  

In [2]:
class Opcode(Enum):
    add = 1
    sub = 2

Tuple documentation

In [3]:
class Format1(Product):
    field_1 = Opcode

class Format2(Product):
    field_1 = Opcode
    field_2 = Opcode

In [4]:
class Instruction(TaggedUnion):
    fmt1 = Format1
    fmt2 = Format2

In [5]:
import itertools as it
def enumerate_adt(t):
    field_specifiers = t.field_dict.keys()
    if issubclass(t, Sum):
        for k,s in t.field_dict.items():
            for v in enumerate_adt(s):
                v_dict = {ki: v if ki == k else None for ki in field_specifiers}
                yield t.from_values(v_dict)
    elif issubclass(t, Tuple):
        iters = []
        for s in t.field_dict.values():
            iters.append(enumerate_adt(s))
        for args in it.product(*iters):
            v_dict = {k: v for k,v in zip(field_specifiers, args)}
            yield t.from_values(v_dict)
    else:
        yield from t.field_dict.values()
        

In [6]:
for i in enumerate_adt(Instruction):
    print(i)

Instruction(fmt1=Format1(field_1=Opcode.add), fmt2=None)
Instruction(fmt1=Format1(field_1=Opcode.sub), fmt2=None)
Instruction(fmt1=None, fmt2=Format2(field_1=Opcode.add, field_2=Opcode.add))
Instruction(fmt1=None, fmt2=Format2(field_1=Opcode.add, field_2=Opcode.sub))
Instruction(fmt1=None, fmt2=Format2(field_1=Opcode.sub, field_2=Opcode.add))
Instruction(fmt1=None, fmt2=Format2(field_1=Opcode.sub, field_2=Opcode.sub))


In [7]:
# class T(enum): ...

# class Ext(enum, extends=T): ... # copy elements of T into Ext

# T < Ext

#from hwtypes import Sum
#T = Sum[hw.Bit, hw.BitVector[4]]
#S = Sum[hw.Bit, hw.BitVector[4], hw.BitVector[6]]
#issubclass(S, T)